In [73]:
from nupack import *
import pandas as pd
import openpyxl
import RNA
import subprocess
import ghostscript
import matplotlib.pyplot as plt


salmonella_16s = "AGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAGCAGCTTGCTGCTTTGCTGACGAGTGGCGGACGGGTGAGTAATGTCTGGGAAACTGCCTGATGGAGGGGGATAACTACTGGAAACGGTGGCTAATACCGCATAACGTCGCAAGACCAAAGAGGGGGACCTTCGGGCCTCTTGCCATCAGATGTGCCCAGATGGGATTAGCTTGTTGGTGAGGTAACGGCTCACCAAGGCGACGATCCCTAGCTGGTCTGAGAGGATGACCAGCCACACTGGAACTGAGACACGGTCCAGACTCCTACGGGAGGCAGCAGTGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCCATGCCGCGTGTATGAAGAAGGCCTTCGGGTTGTAAAGTACTTTCAGCGGGGAGGAAGGTGTTGTGGTTAATAACCGCAGCAATTGACGTTACCCGCAGAAGAAGCACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCACGCAGGCGGTCTGTCAAGTCGGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATTCGAAACTGGCAGGCTTGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGGCCCCCTGGACAAAGACTGACGCTCAGGTGCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGTCTACTTGGAGGTTGTGCCCTTGAGGCGTGGCTTCCGGAGCTAACGCGTTAAGTAGACCGCCTGGGGAGTACGGCCGCAAGGTTAAAACTCAAATGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGATGCAACGCGAAGAACCTTACCTGGTCTTGACATCCACAGAACTTTCCAGAGATGGATTGGTGCCTTCGGGAACTGTGAGACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTTGTGAAATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTATCCTTTGTTGCCAGCGGTCCGGCCGGGAACTCAAAGGAGACTGCCAGTGATAAACTGGAGGAAGGTGGGGATGACGTCAAGTCATCATGGCCCTTACGACCAGGGCTACACACGTGCTACAATGGCGCATACAAAGAGAAGCGACCTCGCGAGAGCAAGCGGACCTCATAAAGTGCGTCGTAGTCCGGATTGGAGTCTGCAACTCGACTCCATGAAGTCGGAATCGCTAGTAATCGTGGATCAGAATGCCACGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCATGGGAGTGGGTTGCAAAAGAAGTAGGTAGCTTAACCTTCGGGGGGGCGCTTACCACTTTGTGATTCATGACTGGGGTGAAGTCGTAACAAGGTAACCGTAGGGGAACCTGCGGTTGGATCACCTCCTT" 

conserved_hairpin_sequence = "GGACUUUAGAACAGAGGAGAUAAAGAUG"
linker = "AACCUGGCGGCAGCGCAAAAG"

# Targeting Series B toehold
# toehold size = 36 nt
# Alpha toehold size = 25
# Betta toehold size = 11

toehold_window = 36

s = salmonella_16s.replace("T", "U")
print(s)

AGAGUUUGAUCAUGGCUCAGAUUGAACGCUGGCGGCAGGCCUAACACAUGCAAGUCGAACGGUAACAGGAAGCAGCUUGCUGCUUUGCUGACGAGUGGCGGACGGGUGAGUAAUGUCUGGGAAACUGCCUGAUGGAGGGGGAUAACUACUGGAAACGGUGGCUAAUACCGCAUAACGUCGCAAGACCAAAGAGGGGGACCUUCGGGCCUCUUGCCAUCAGAUGUGCCCAGAUGGGAUUAGCUUGUUGGUGAGGUAACGGCUCACCAAGGCGACGAUCCCUAGCUGGUCUGAGAGGAUGACCAGCCACACUGGAACUGAGACACGGUCCAGACUCCUACGGGAGGCAGCAGUGGGGAAUAUUGCACAAUGGGCGCAAGCCUGAUGCAGCCAUGCCGCGUGUAUGAAGAAGGCCUUCGGGUUGUAAAGUACUUUCAGCGGGGAGGAAGGUGUUGUGGUUAAUAACCGCAGCAAUUGACGUUACCCGCAGAAGAAGCACCGGCUAACUCCGUGCCAGCAGCCGCGGUAAUACGGAGGGUGCAAGCGUUAAUCGGAAUUACUGGGCGUAAAGCGCACGCAGGCGGUCUGUCAAGUCGGAUGUGAAAUCCCCGGGCUCAACCUGGGAACUGCAUUCGAAACUGGCAGGCUUGAGUCUUGUAGAGGGGGGUAGAAUUCCAGGUGUAGCGGUGAAAUGCGUAGAGAUCUGGAGGAAUACCGGUGGCGAAGGCGGCCCCCUGGACAAAGACUGACGCUCAGGUGCGAAAGCGUGGGGAGCAAACAGGAUUAGAUACCCUGGUAGUCCACGCCGUAAACGAUGUCUACUUGGAGGUUGUGCCCUUGAGGCGUGGCUUCCGGAGCUAACGCGUUAAGUAGACCGCCUGGGGAGUACGGCCGCAAGGUUAAAACUCAAAUGAAUUGACGGGGGCCCGCACAAGCGGUGGAGCAUGUGGUUUAAUUCGAUGCAACGCGAAGAACCUUACCUGGUCUUGACAUCCACAGAACU

In [71]:
def screen_rna(rna:str, toehold_window):
    struct_mfe_container = []
    stop_codons = ["UGA", "UAG", "UAA"]
    fail_list = []
    succeed_list = []
    outer_defect_dict = {}
    rna = rna.replace("T", "U")
    for i in range(len(rna)):
    # for i in range(300):
        if i + toehold_window < len(rna):
            trigger_strand = Strand(rna[i:i+toehold_window], name="trigger")
            toehold_strand = Strand(str(~trigger_strand), name="toehold")
            toehold_betta_compliment = ~Strand(str(toehold_strand)[-11:], name="~ toehold betta")
            check_stop = any(stop in str(toehold_betta_compliment) for stop in stop_codons)
            if not check_stop:
                try:
                    # Creating domains:
                    trigger_domain = Domain(str(trigger_strand), name="trigger domain")
                    toehold_domain = Domain(str(toehold_strand), name="toehold domain")
                    hairpin_domain = Domain(conserved_hairpin_sequence, name="hairpin")
                    toehold_betta_compliment_domain = Domain(str(toehold_betta_compliment), name="toehold betta compliment domain")
                    linker_domain = Domain(linker, name="linker domain")

                    # Creating Target Strands:
                    switch_target_strand = TargetStrand([toehold_domain, hairpin_domain, toehold_betta_compliment_domain, linker_domain], name="Switch Target Strand")
                    switch_target_strand_str = toehold_domain.__str__() + hairpin_domain.__str__() + toehold_betta_compliment_domain.__str__() + linker_domain.__str__()
                    trigger_target_strand = TargetStrand([trigger_domain], name="Trigger Target Strand")
                    trigger_target_strand_str = trigger_domain.__str__()
                    # print(i, "Switch: ", switch_target_strand_str, "Trigger: ", trigger_target_strand_str)
                    
                    struct_mfe_folding = RNA.cofold(switch_target_strand_str + "&" + trigger_target_strand_str)
                    # struct_mfe_folding = RNA.cofold(switch_target_strand_str + "&" + salmonella_16s)

                    hybridized_seq = switch_target_strand_str + trigger_target_strand_str
                    # hybridized_seq = switch_target_strand_str + salmonella_16s

                    struct_mfe_folding.append(hybridized_seq)
                    struct_mfe_folding.append(len(struct_mfe_container))
                    struct_mfe_container.append(struct_mfe_folding)
                    
                    # Creating Complexes:
                    switch_complex = TargetComplex([switch_target_strand], ".25(11.3(5.12)5.3)11.21", name="Switch Complex")
                    trigger_complex = TargetComplex([trigger_target_strand], ".36", name="Trigger Complex")
                    switch_trigger_complex = TargetComplex([trigger_target_strand, switch_target_strand], "(36+)36.60") # .3(5.12)5.3)11.21

                    # Creating Tubes:
                    Tube1 = TargetTube(on_targets={switch_complex:1e-8, trigger_complex:1e-8, switch_trigger_complex: 1e-8}, name='Tube1',  off_targets=SetSpec(max_size=3, include=[], exclude=[]))
                    my_model = Model()
                    my_tubes = [Tube1]
                    my_design = tube_design(tubes=my_tubes,
                        hard_constraints=[], soft_constraints=[],
                        defect_weights=None, options=None, model=my_model)   

                    # Running Tube
                    results = my_design.run(trials=1)[0]
                    x = pd.DataFrame(results.to_analysis)
                    y = pd.DataFrame(results.concentrations.table)
                    defect_df_z = pd.DataFrame(results.defects.complexes) 
                    defect_dict = results.defects.complexes
                    g = pd.concat([x, y, defect_df_z])
                    outer_defect_dict[switch_target_strand_str] = defect_df_z.to_dict()
                    succeed_list.append(switch_target_strand_str)

                    #Export to Excel
                    # os.chdir(r"/home/ORISE/")
                    # print(os.getcwd())
                    # with pd.ExcelWriter("test.xlsx", mode="a", engine="openpyxl") as writer:
                    #     g.to_excel(writer, sheet_name=str(i))   

                except:
                    print(str(switch_target_strand_str) + " failed")
                    fail_list.append(str(trigger_strand))
    print("Suceeded: ", len(fail_list))
    print("Failed: ", len(succeed_list))
    return struct_mfe_container, outer_defect_dict               



struct_mfe, defect_dict = screen_rna(salmonella_16s, toehold_window)

7 Switch:  UAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAUGAUCGGACUUUAGAACAGAGGAGAUAAAGAUGGAUCAUGGCUCAACCUGGCGGCAGCGCAAAAG Trigger:  GAUCAUGGCUCAGAUUGAACGCUGGCGGCAGGCCUA
8 Switch:  UUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAUGAUGGACUUUAGAACAGAGGAGAUAAAGAUGAUCAUGGCUCAAACCUGGCGGCAGCGCAAAAG Trigger:  AUCAUGGCUCAGAUUGAACGCUGGCGGCAGGCCUAA
9 Switch:  GUUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAUGAGGACUUUAGAACAGAGGAGAUAAAGAUGUCAUGGCUCAGAACCUGGCGGCAGCGCAAAAG Trigger:  UCAUGGCUCAGAUUGAACGCUGGCGGCAGGCCUAAC
10 Switch:  UGUUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAUGGGACUUUAGAACAGAGGAGAUAAAGAUGCAUGGCUCAGAAACCUGGCGGCAGCGCAAAAG Trigger:  CAUGGCUCAGAUUGAACGCUGGCGGCAGGCCUAACA
11 Switch:  GUGUUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAUGGACUUUAGAACAGAGGAGAUAAAGAUGAUGGCUCAGAUAACCUGGCGGCAGCGCAAAAG Trigger:  AUGGCUCAGAUUGAACGCUGGCGGCAGGCCUAACAC
12 Switch:  UGUGUUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCAGGACUUUAGAACAGAGGAGAUAAAGAUGUGGCUCAGAUUAACCUGGCGGCAGCGCAAAAG Trigger:  UGGCUCAGAUUGAACGCUGGCGGCAGGCCUAACACA
13 Switch:  AUGUGUUAGGCCUGCCGCCAGCGUUCAAUCUGAGCCGGACUUUAGAACAGAGGAG

In [72]:
modified_defects = defect_dict.copy()

for i in modified_defects:
    modified_defects[i]["nominal defect sum"] = modified_defects[i]["defect"][0] +  modified_defects[i]["defect"][1] + modified_defects[i]["defect"][2]
    modified_defects[i]["normalized defect sum"] = modified_defects[i]["normalized"][0] +  modified_defects[i]["normalized"][1] + modified_defects[i]["normalized"][2]
    # print(modified_defects[i]["normalized defect sum"])


from collections import OrderedDict
from operator import getitem

modified_defects_sort = OrderedDict(sorted(modified_defects.items(),
       key = lambda x: getitem(x[1], "normalized defect sum")))

for i in modified_defects_sort.keys():
    print(modified_defects_sort[i]["normalized defect sum"], i)
    

0.6123237328435132 UAUCUAAUCCUGUUUGCUCCCCACGCUUUCGCACCUGGACUUUAGAACAGAGGAGAUAAAGAUGAGGUGCGAAAGAACCUGGCGGCAGCGCAAAAG
0.6150336830570836 GUAUCUAAUCCUGUUUGCUCCCCACGCUUUCGCACCGGACUUUAGAACAGAGGAGAUAAAGAUGGGUGCGAAAGCAACCUGGCGGCAGCGCAAAAG
0.6328843262802365 GUUUCCAGUAGUUAUCCCCCUCCAUCAGGCAGUUUCGGACUUUAGAACAGAGGAGAUAAAGAUGGAAACUGCCUGAACCUGGCGGCAGCGCAAAAG
0.6375916406130435 CAUUUCACCGCUACACCUGGAAUUCUACCCCCCUCUGGACUUUAGAACAGAGGAGAUAAAGAUGAGAGGGGGGUAAACCUGGCGGCAGCGCAAAAG
0.641371226808842 AAGCUGCUUCCUGUUACCGUUCGACUUGCAUGUGUUGGACUUUAGAACAGAGGAGAUAAAGAUGAACACAUGCAAAACCUGGCGGCAGCGCAAAAG
0.6498292171334288 UUUCCAGUAGUUAUCCCCCUCCAUCAGGCAGUUUCCGGACUUUAGAACAGAGGAGAUAAAGAUGGGAAACUGCCUAACCUGGCGGCAGCGCAAAAG
0.6529913847136353 AUCUAAUCCUGUUUGCUCCCCACGCUUUCGCACCUGGGACUUUAGAACAGAGGAGAUAAAGAUGCAGGUGCGAAAAACCUGGCGGCAGCGCAAAAG
0.656147594145233 GGUAUCUAAUCCUGUUUGCUCCCCACGCUUUCGCACGGACUUUAGAACAGAGGAGAUAAAGAUGGUGCGAAAGCGAACCUGGCGGCAGCGCAAAAG
0.6605106421896627 GGAAAGUUCUGUGGAUGUCAAGACCAGGUAAGGUUCGGACUUUAGAACAGAGGAG